# Column
id, title, author, genre, story, platform, age, rate, free, complete, thumbnail, website
    - id: 인덱스
    - title: 제목
    - author: 작가
    - genre: 장르
    - story: 줄거리
    - platform: 플랫폼 (네이버, 다음, 카카오, 레진코믹스 중 1)
    - age: 연령대
    - rate: 평점
    - free: 유료 여부 (0: 유료, 1: 무료)
    - complete: 완결여부
    - thumbnail: 썸네일
    - website: 웹 주소

## 연재 웹툰

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

import pandas as pd

url = 'http://webtoon.daum.net/'
week = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
week_match = '#day=&tab=day'

# 로그인 정보
email = ''
pw = ''

title = []
total_tc = 0
now_tc = 0 

author = {}
genre = {}
story = {}
platform = {}
age = {}
rate = {}
cuttoon = {}
thumbnail = {}
website = {}


for d in week:
    link = url + week_match[0:5] + d + week_match[5:] 
    #print(link)
    
    # webdriver_manager를 사용해 execution_path 변수를 설정
    # 스크립트를 실행할 때 자동으로 드라이버 다운로드
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(link)
    
    # 개발자도구에서 보이는 source code 가져옴
    html = driver.page_source
    bsObject = BeautifulSoup(html, 'html.parser')

    # title, age
    titles = bsObject.find_all('strong', {'class':'tit_wt'}) # find_all('태그명', {'속성명': '값' ...})
    now_tc = len(titles)
    print(f'now_tc={now_tc}')
    for t in titles:
        adult = t.find('span', {'class': "ico_comm ico_adult"})
        
        if adult is not None: # 성인물일 경우
            idx = t.text[2:]
            title.append(idx)
            age[idx] = 19
            #print(f'19 {idx}')
       
        else: # 연령제한이 없는 경우
            idx = t.text    
            title.append(idx)
            age[idx] = 0
            #print(t.text)
        
    # author
    authors = bsObject.find_all('span', {'class':'txt_info'})
    for a in enumerate(authors):
        idx, name = a
        author[title[total_tc+idx]] = name.text[2:]
        #print(name.text[2:])
    
    # thumbnail
    thumbnails = bsObject.find_all('img', {'class': 'img_thumb'})
    for i in range(5,now_tc+5): # 0-4: 배너에 떠있는 웹툰
        idx = i-5
        thumb = thumbnails[i]['src']
        thumbnail[title[total_tc + idx]] = thumb
        #print(title[total_tc + idx], thumb)
    
    
    # 세부 주소 저장
    newlink_match = bsObject.find_all('a', {'class': 'link_wt'})
    for idx in range(0, now_tc):
        newlink = url[:-1] + newlink_match[idx]['href']
        website[title[total_tc+idx]] = newlink
        print(title[total_tc+idx], newlink)
        
    # 세부 주소으로 이동 (장르, 줄거리, 평점)
    for pIdx in range(0, now_tc):
        login_check = False

        page = WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'link_wt')))[:-10]
        print(d, pIdx, len(page))
        driver.execute_script("arguments[0].click();", page[pIdx])
        
        link = driver.current_url

        # 성인물인 경우 로그인이 필요함
        if link[8:14] == "logins":
            login_check = True
            driver.find_element_by_css_selector('#mArticle > div.section_login > div > div > div.login_account > a.link_login.link_klogin').click()
            driver.find_element_by_name('email').send_keys(email)
            driver.find_element_by_name('password').send_keys(pw)
            driver.find_element_by_css_selector('#login-form > fieldset > div.wrap_btn > button.btn_g.btn_confirm.submit').click()

        # 다 뜰때까지 존버
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'txt_genre')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'txt_story')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'emph_grade')))
        
        html =  driver.page_source
        bsObject = BeautifulSoup(html, 'html.parser')

        # 장르
        genres = bsObject.find('dd', {'class': 'txt_genre'}).text.strip()
        genre[title[total_tc + pIdx]] = genres
        #print(genre)

        # 줄거리
        stories = bsObject.find('dd', {'class' : 'txt_story'}).text
        story[title[total_tc + pIdx]] = stories
        #print(story)

        # 평점
        rates = bsObject.find('em', {'class':'emph_grade'})
        rate[title[total_tc + pIdx]] = rates.text
        #print(rates.text)

        if login_check:
            driver.back()
            driver.back()
        driver.back()
        
        # 다 뜰때까지 존버
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'tit_wt')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'txt_info')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'img_thumb')))
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'link_wt')))
    
    total_tc += now_tc



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\gunso\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


now_tc=23


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [2]:
print(len(title), len(author), len(genre), len(story), len(age), len(rate), len(thumbnail), len(website))

148 145 145 145 145 145 145 145


In [3]:
daum_webtoon = pd.DataFrame(columns = ['id', 'title', 'author', 'genre', 'platform', 'story', 'age', 'rate', 'complete', 'thumbnail', 'website'])
daum_webtoon['title'] = list(set(title)) # 중복 제거
daum_webtoon.set_index(['title'], inplace = True)
daum_webtoon.head(5)

,id,author,genre,platform,story,age,rate,complete,thumbnail,website
title,,,,,,,,,,
저기요 손님,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
우리가 사랑하는 방법,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
대선비쇼에 어서오세요!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
지옥사원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
학사재생,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
col = ['author', 'genre', 'story', 'age', 'rate', 'thumbnail', 'website']
dic = [author, genre, story, age, rate, thumbnail, website]
for i in range(0, len(dic)):
    for key, value in dic[i].items():
         daum_webtoon.loc[key][col[i]] = value
daum_webtoon.head(1)

,id,author,genre,platform,story,age,rate,complete,thumbnail,website
title,,,,,,,,,,
저기요 손님,NaN,강소소,"드라마, 코믹, 친구",NaN,너무 다른 네 명의 절친이 뜬금없이 카페 동업하면서 벌어지는 좌충우돌 청춘드라마,0,9.5,NaN,http://t1.daumcdn.net/webtoon/op/8202c8be50f1c...,http://webtoon.daum.net/webtoon/view/excuseme


In [5]:
daum_webtoon['complete'] = 0
daum_webtoon['platform'] = '다음 웹툰'
daum_webtoon['free'] = 1
daum_webtoon = daum_webtoon.reset_index().rename(columns={"index": ""})
daum_webtoon = daum_webtoon[['id', 'title', 'author', 'genre', 'platform', 'story', 'age', 'rate', 'free', 'complete', 'thumbnail', 'website']]

In [6]:
daum_webtoon.head(1)

,id,title,author,genre,platform,story,age,rate,free,complete,thumbnail,website
0,NaN,저기요 손님,강소소,"드라마, 코믹, 친구",다음 웹툰,너무 다른 네 명의 절친이 뜬금없이 카페 동업하면서 벌어지는 좌충우돌 청춘드라마,0,9.5,1,0,http://t1.daumcdn.net/webtoon/op/8202c8be50f1c...,http://webtoon.daum.net/webtoon/view/excuseme


In [7]:
daum_webtoon.to_csv("daum_webtoon.csv", mode='w', encoding = 'euc-kr')